In [30]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
import netgen.geom2d as geom2d
from ngsolve.krylovspace import CGSolver
from ngsolve import VTKOutput


In [31]:
# points for mesh vertices
geo = geom2d.SplineGeometry()
p1 = geo.AppendPoint (0.0, 0.0)
p2 = geo.AppendPoint (1.0, 0.0)
p3 = geo.AppendPoint (1.0, 0.5)
p4 = geo.AppendPoint (1.0, 1.0)
p5 = geo.AppendPoint (0.0, 1.0)

# segments for the boundaries and bcs labels
geo.Append (["line", p1, p2], bc="bottom")
geo.Append (["line", p2, p3], bc="low_right")
geo.Append (["line", p3, p4], bc="top_right")
geo.Append (["line", p4, p5], bc="top")
geo.Append (["line", p5, p1], bc="left")

# actual mesh generation:
# generate a mesh using points
mesh = geo.GenerateMesh(maxh=0.05)
mesh.Curve(3)

# create a NGSolve mesh class 
mesh = Mesh(mesh)

# scene = Draw(mesh)

In [32]:
# physical parameters
mu = 1.0
lam = 1.25

# definition of the Piola-Kirchhoff stress tensor
def Stress(strain):
    return 2.0*mu*strain + lam*Trace(strain)*Id(mesh.dim)   

In [33]:
# since we want to impose Dirichlet bcs on components of u, we build two scalar H1 FESpaces
# and glue them together in a compound FESpace. 

# polynomial degree of the FESpaces
poly_degree = 3

# scalar H1 spaces for each component of the displacement
fesx = H1(mesh, order=poly_degree, dirichlet="low_right")
fesy = H1(mesh, order=poly_degree, dirichlet="bottom")

# compound FESpace for the vector valued displacement and GridFunction for the solution
fes = FESpace([fesx, fesy])
gfu = GridFunction(fes)

# solution and test functions: we declare explicitly the components since
# the FESpaces are scalar
(ux, uy), (vx, vy) = fes.TnT()

# u and v are the vector valued CoefficientFunctions
u = CoefficientFunction((ux, uy))
v = CoefficientFunction((vx, vy))

# we need the gradients to compute the strain and the stress tensors.
# gradu and gradv are matrix valued CoefficientFunctions
gradu = CoefficientFunction( (Grad(ux), Grad(uy)), dims=(mesh.dim,mesh.dim) )
gradv = CoefficientFunction( (Grad(vx), Grad(vy)), dims=(mesh.dim,mesh.dim) )

# bilinear form a(.,.)
with TaskManager():
    a = BilinearForm(InnerProduct(Stress(Sym(gradu)), Sym(gradv)).Compile()*dx)
    pre = Preconditioner(a, "bddc")
    a.Assemble()

# linear form f(.): we apply a horizontal traction on the left edge
force = CF( (-0.2, 0.0) )
f = LinearForm(force*v*ds("left")).Assemble()

In [34]:
# solve the linear system
inv = CGSolver(a.mat, pre, tol=1e-8)
gfu.vec.data = inv * f.vec

# plotting the solution on the deformed mesh using the webgui Draw.
# We must use CoefficientFunction and separate the components of gfu
# since gfu is a vector valued GridFunction from a compound FESpace. 
# Draw does not accept GridFunction in this case.
Draw(CoefficientFunction((gfu.components[0], gfu.components[1])), mesh, deformation=CoefficientFunction((gfu.components[0], gfu.components[1])))

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [35]:
# export solution to VTK

# VTKOutput in NGSolve exports only the linear geometry even if the mesh is curved, 
# so we use the subdivision option to get a smoother representation of the curved geometry.

# Remark: we subdivide the elements only for visualization, not for computations. This means
# that the solution is still piecewise polynomial of degree poly_degree on each element, 
# but in the .vtu file the elements are subdivided in smaller linear elements for a better 
# visualization of the  curved geometry. This increases the number of points in the .vtu file, 
# increasing its size, but not affecting the accuracy of the solution.
vtk = VTKOutput(
    mesh,
    coefs=[gfu.components[0], gfu.components[1]],
    names=["u_x", "u_y"],
    filename="solution",
    subdivision=3  # or higher for smoother curves
)
vtk.Do()

'solution'